# VANNA AI Demo with NASA Oct 22-23 IT Spend Data (from Sam.gov)

Here we experiment with LLM generation of SQL through the open source VANNA project (vanna.ai).

We begin with a real world problem: Tracking the IT Spend of government agencies. Specifically, there are two code systems in use, NAICS and PSC. So here we will use Vanna to compare IT spending according to NAICS codes and PSC codes by NASA from October 2022 to 2023.

Specifically, IT spending according to NAICS codes begins with "54", while for PSC codes begins with "D". So we pulled the data for NASA from Sam.gov twice, once for NAICS codes that begin with "54" and once for PSC codes that begin with "D". We will then use Vanna to look at the overlap and differences in the data.

## First let's import the CSV files into a local sqlite database

In [2]:
import csv
import sqlite3

# Connect to a local file
connection = sqlite3.connect('nasa_spend_10_22_23.db')
cursor = connection.cursor()

In [3]:
# Create the NASA NAICS IT SPEND table
cursor.execute('''
CREATE TABLE nasa_naics_it_spend(
 id INTEGER PRIMARY KEY AUTOINCREMENT,
 contracting_depratment_id TEXT,
 contracting_department_name TEXT,
 contracting_office_id TEXT,
 contracting_office_name TEXT,
 naics_code TEXT,
 naics_description TEXT,
 psc_code TEXT,
 psc_description TEXT,
 description_of_requirement TEXT,
 ultimate_parent_legal_business_name TEXT,
 ultimate_parent_unique_entity_id TEXT,
 dollars_obligated REAL);
''')

In [4]:
# open the NAICS IT SPEND CSV file
file = open('NASA_NAICS_IT_SPEND_Oct_22_to_23.csv')
contents = csv.reader(file)
next(contents, None) # skip the header
insert_records = '''INSERT INTO nasa_naics_it_spend (contracting_depratment_id, contracting_department_name, 
 contracting_office_id, contracting_office_name, naics_code, naics_description, psc_code, psc_description,
 description_of_requirement, ultimate_parent_legal_business_name, ultimate_parent_unique_entity_id,
 dollars_obligated) values (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
'''
cursor.executemany(insert_records,contents)

In [5]:
# Test by summing up all expenses
rows = cursor.execute("select sum(dollars_obligated) from nasa_naics_it_spend").fetchall()
rows

[(1054609028.0799993,)]

In [6]:
# Create the NASA PSC IT SPEND table
cursor.execute('''
CREATE TABLE nasa_psc_it_spend(
 id INTEGER PRIMARY KEY AUTOINCREMENT,
 contracting_depratment_id TEXT,
 contracting_department_name TEXT,
 contracting_office_id TEXT,
 contracting_office_name TEXT,
 naics_code TEXT,
 naics_description TEXT,
 psc_code TEXT,
 psc_description TEXT,
 description_of_requirement TEXT,
 ultimate_parent_legal_business_name TEXT,
 ultimate_parent_unique_entity_id TEXT,
 dollars_obligated REAL);
''')

In [7]:
# open the PSC IT SPEND CSV file
file = open('NASA_PSC_IT_SPEND_Oct_22_to_23.csv')
contents = csv.reader(file)
next(contents, None) # skip the header
insert_records = '''INSERT INTO nasa_psc_it_spend (contracting_depratment_id, contracting_department_name, 
 contracting_office_id, contracting_office_name, naics_code, naics_description, psc_code, psc_description,
 description_of_requirement, ultimate_parent_legal_business_name, ultimate_parent_unique_entity_id,
 dollars_obligated) values (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
'''
cursor.executemany(insert_records,contents)

In [8]:
# Test by summing up all expenses
rows = cursor.execute("select sum(dollars_obligated) from nasa_psc_it_spend").fetchall()
rows

[(1119386991.0599995,)]

# So now we have a database. Let's connect to Vanna and teach it about our system